# Preprocessing
This notebook filters the data and saves training and test data in the data folder

In [13]:
# import basic libraries
import pandas as pd

# import pm4py library to work with XES logs and process mining
import pm4py

# for label encoding
from sklearn.preprocessing import LabelEncoder

In [14]:
log = pm4py.read_xes("data/BPI_Challenge_2017.xes.gz")
log_df = pm4py.convert_to_dataframe(log)
log_df.head()

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Event index in trace

We add to each event in the log its position in their trace

In [15]:
# add column "event_index_in_trace"
# which indicates the 1st, 2nd ... event in the trace
log_df = log_df.sort_values(by=["case:concept:name", "time:timestamp"])
log_df["event_index_in_trace"] = log_df.groupby("case:concept:name").cumcount()

## Remaining time

Here we calculate the remaining time per trace

In [16]:
# which indicates time from that event until the last event in the trace
log_df["time:timestamp"] = pd.to_datetime(log_df["time:timestamp"], utc=True)
log_df["remaining_time"] = log_df.groupby("case:concept:name")["time:timestamp"].transform(lambda x: x.max() - x).dt.total_seconds() / (24 * 60 * 60)  # convert to float days

## Time being executed

We also will store the time that the trace is being executed from the first event in the trace until the current event.

In [17]:
log_df["time:timestamp"] = pd.to_datetime(log_df["time:timestamp"], utc=True)

# Time in execution: time that has been the trace in execution from the first event
log_df["execution_time"] = log_df.groupby("case:concept:name")["time:timestamp"].transform(lambda x: x - x.min()).dt.total_seconds() / (24 * 60 * 60)

## Construct the data 5 prefix length

The idea is to construct a database with traces of length 5. The information is gathered by joinning the 5 events of traces that happens following to each other until the end of the trace

## Feature encoding

For now we use the basic feature encoding from pm4py, but we want to experiment with using complex index encoding, where we encode the previous 10 activities (or add padding). Furthermore, we add the index of the activity in the log

In [18]:
# select the features we are going to encode
columns_to_onehot = ['Action', 'concept:name', 'case:LoanGoal',]
columns_to_label = ['org:resource','EventOrigin','lifecycle:transition','case:ApplicationType','case:concept:name']

# label-encode the data
label_encoder = LabelEncoder()
#log_df_encode = label_encoder.fit_transform(log_df[columns_to_label])
log_df[columns_to_label] = log_df[columns_to_label].apply(lambda col: label_encoder.fit_transform(col))


# one-hot encode the data
log_df_encode = pd.get_dummies(log_df[columns_to_onehot], dtype=int)
log_df_encode.head(3)

,Action_Created,Action_Deleted,Action_Obtained,Action_Released,Action_statechange,concept:name_A_Accepted,concept:name_A_Cancelled,concept:name_A_Complete,concept:name_A_Concept,concept:name_A_Create Application,...,case:LoanGoal_Debt restructuring,case:LoanGoal_Existing loan takeover,case:LoanGoal_Extra spending limit,case:LoanGoal_Home improvement,case:LoanGoal_Motorcycle,case:LoanGoal_Not speficied,"case:LoanGoal_Other, see explanation",case:LoanGoal_Remaining debt home,case:LoanGoal_Tax payments,case:LoanGoal_Unknown
686058,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
686059,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
686060,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
columns_to_keep = ['EventID','time:timestamp', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount',
       'event_index_in_trace', 'remaining_time', 'execution_time',
        'org:resource','EventOrigin','lifecycle:transition',
                   'case:ApplicationType','case:concept:name']

# Concatenate the DataFrames based on the index
log_df = pd.concat([log_df[columns_to_keep], log_df_encode], axis=1)

log_df.head()

,EventID,time:timestamp,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,MonthlyCost,Selected,CreditScore,OfferedAmount,event_index_in_trace,...,case:LoanGoal_Debt restructuring,case:LoanGoal_Existing loan takeover,case:LoanGoal_Extra spending limit,case:LoanGoal_Home improvement,case:LoanGoal_Motorcycle,case:LoanGoal_Not speficied,"case:LoanGoal_Other, see explanation",case:LoanGoal_Remaining debt home,case:LoanGoal_Tax payments,case:LoanGoal_Unknown
686058,Application_1000086665,2016-08-03 15:57:21.673000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,1,0,0,0
686059,ApplState_161925113,2016-08-03 15:57:21.734000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,1,0,0,0
686060,Workitem_747707399,2016-08-03 15:57:21.963000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,1,0,0,0
686061,Workitem_1030261128,2016-08-03 15:58:28.286000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,3,...,0,0,0,0,0,0,1,0,0,0
686062,Workitem_1127124826,2016-08-03 15:58:28.293000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,4,...,0,0,0,0,0,0,1,0,0,0


## Split train and test

Using the pm4py.split_train_test resulted in traces in train that ended after the start of traces in test unfortunately. This is not a good split, so we implement it manually by sorting traces on timestamp

In [20]:
log_df = log_df.sort_values(by=["case:concept:name", "time:timestamp"])
trace_start_df = log_df[["case:concept:name", "time:timestamp"]].groupby(["case:concept:name"]).min()
trace_end_df = log_df[["case:concept:name", "time:timestamp"]].groupby(["case:concept:name"]).max()

In [21]:
# take the last 10% of the traces as test set
test_size = round(len(trace_start_df)*0.1)
test_cases = trace_start_df.sort_values("time:timestamp").tail(test_size)

In [22]:
# train cases must end before test cases start
train_cases = trace_end_df[trace_end_df["time:timestamp"] < test_cases["time:timestamp"].min()]

In [23]:
train_df = log_df[log_df["case:concept:name"].isin(train_cases.index)]
test_df = log_df[log_df["case:concept:name"].isin(test_cases.index)]

In [24]:
# double check that the timestamps don't overlap
# all traces in train must end before the start of traces in test
print(train_df["time:timestamp"].max())
print(test_df["time:timestamp"].min())

2016-11-22 09:21:30.939000+00:00
2016-11-22 09:22:17.274000+00:00


## Random function

In [ ]:
# Input: an event = a row from the dataframe, data_log = if it is from the X_train or X_test (this way we ensure to to leak data)
def curr_workload(event, data_log):
    curr_time = event['time:timestamp']
    curr_resource = event['org:resource']

    df_resource = data_log[(data_log['org:resource'] == curr_resource) & (data_log['time:timestamp'] <= curr_time)]

    if df_resource.empty:
        return curr_resource, -1
    return max(n, 0)
    n = len(df_resource)
    # Now, in df_resource we have the events with the resource that has participated before time i.
    # But, have all the events ongoing? Or have already finished?

    df_resource_sorted = df_resource.sort_values(by='event_index_in_trace')
    last_events = df_resource_sorted.groupby('case:concept:name').apply(lambda group: group.iloc[-1])

    # Check the last lifecycle transition for each activity (to know if it is finished or suspended)
    for _, last_ev in last_events.iterrows():
        if last_ev['lifecycle:transition'] in ['complete', 'schedule', 'withdraw', 'ate_abort']:
            n -= len(df_resource[df_resource['case:concept:name'] == last_ev['case:concept:name']])

    return max(n, 0)

## Save features X and targets y of train and test

In [25]:
#Save the one-hot encoded dataframes
X_train = train_df.drop(columns=["remaining_time"])
X_train.to_csv("data/generated/onehot/X_train.csv")

X_test = test_df.drop(columns=["remaining_time"])
X_test.to_csv("data/generated/onehot/X_test.csv")

y_train = train_df["remaining_time"]
y_train.to_csv("data/generated/onehot/y_train.csv")

y_test = test_df["remaining_time"]
y_test.to_csv("data/generated/onehot/y_test.csv")

## Frequency encoding and save train test files


In [26]:
# Create frequency encoding for test and train df
# Select columns that start with "concept:" or "Action_"
relevant_columns = [c for c in log_df_encode.columns if c.startswith("concept:") or c.startswith("Action_")]

for trace_id, trace_df in train_df.groupby("case:concept:name"):
    trace_df_sorted = trace_df.sort_values(by='event_index_in_trace')
    # Update only the selected columns with the cumulative sum
    train_df.loc[trace_df_sorted.index, relevant_columns] = trace_df_sorted[relevant_columns].cumsum()

In [27]:
# Select columns that start with "concept:" or "Action_"
relevant_columns = [c for c in log_df_encode.columns if c.startswith("concept:") or c.startswith("Action_")]

for trace_id, trace_df in test_df.groupby("case:concept:name"):
    trace_df_sorted = trace_df.sort_values(by='event_index_in_trace')
    # Update only the selected columns with the cumulative sum
    test_df.loc[trace_df_sorted.index, relevant_columns] = trace_df_sorted[relevant_columns].cumsum()

## Save features X and targets y of train and test

In [28]:
#Save the frequency encoded dataframes
X_train = train_df.drop(columns=["remaining_time"])
X_train.to_csv("data/generated/frequency/X_train.csv")

X_test = test_df.drop(columns=["remaining_time"])
X_test.to_csv("data/generated/frequency/X_test.csv")

y_train = train_df["remaining_time"]
y_train.to_csv("data/generated/frequency/y_train.csv")

y_test = test_df["remaining_time"]
y_test.to_csv("data/generated/frequency/y_test.csv")

In [29]:
X_train.head()

,EventID,time:timestamp,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,MonthlyCost,Selected,CreditScore,OfferedAmount,event_index_in_trace,...,case:LoanGoal_Debt restructuring,case:LoanGoal_Existing loan takeover,case:LoanGoal_Extra spending limit,case:LoanGoal_Home improvement,case:LoanGoal_Motorcycle,case:LoanGoal_Not speficied,"case:LoanGoal_Other, see explanation",case:LoanGoal_Remaining debt home,case:LoanGoal_Tax payments,case:LoanGoal_Unknown
686058,Application_1000086665,2016-08-03 15:57:21.673000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,1,0,0,0
686059,ApplState_161925113,2016-08-03 15:57:21.734000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,1,0,0,0
686060,Workitem_747707399,2016-08-03 15:57:21.963000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,1,0,0,0
686061,Workitem_1030261128,2016-08-03 15:58:28.286000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,3,...,0,0,0,0,0,0,1,0,0,0
686062,Workitem_1127124826,2016-08-03 15:58:28.293000+00:00,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,4,...,0,0,0,0,0,0,1,0,0,0
